In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('redhen_preprocessed.csv')
data.head()

,Unnamed: 0,StoryID,Excerpt,CodesApplied_Combined,ACCOUNT,ACCOUNT_Cultural,ACCOUNT_Individual,ACCOUNT_Other,COMMUNITYRECOVERY,EVENT,...,TRAUMA_Societal,VICTIMS,word_count,char_count,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean
0,7992,NI3079,Elliot Rodger Wangs roommate stabbed Wang and ...,EVENT,0,0,0,0,99,1,...,99,0,34,215.0,elliot rodger wangs roommate stabbed wang two ...,"['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'..."
1,4686,NI2689,Because the tragedy unfolded at the start of a...,RESOURCES,0,0,0,0,99,0,...,99,0,32,228.0,tragedy unfolded start holiday weekend school ...,"['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee..."
2,6130,NI2400,Gunrelated restraining orders Also passed in r...,"POLICY, POLICY- Guns, POLICY- Mental health",0,0,0,0,99,0,...,99,0,122,734.0,gunrelated restraining orders also passed resp...,"['gunrelated', 'restraining', 'orders', 'also'...","['gunrelated', 'restraining', 'orders', 'also'...","['gunrel', 'restrain', 'order', 'also', 'pass'...","['gunrelated', 'restraining', 'order', 'also',...","['gunrel', 'restrain', 'order', 'also', 'pass'..."
3,5876,NI1393,Linder said she will also remember Weiss for h...,"VICTIMS, GRIEF/LOSS",0,0,0,0,99,0,...,99,1,100,499.0,linder said also remember weiss epic shot pool...,"['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'rememb', 'weiss', ...","['linder', 'said', 'also', 'remember', 'wei', ...","['linder', 'said', 'also', 'rememb', 'weiss', ..."
4,4126,NI3259,I mean I know how this goes We all do Werent y...,"ACCOUNTABILITY, ACCOUNT- Culture/societal risk...",1,1,0,0,99,0,...,99,0,85,425.0,mean know goes werent sort expecting father on...,"['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...","['mean', 'know', 'go', 'werent', 'sort', 'expe...","['mean', 'know', 'goe', 'werent', 'sort', 'exp..."


In [4]:
data.shape

(8131, 62)

In [5]:
data.columns

Index(['Unnamed: 0', 'StoryID', 'Excerpt', 'CodesApplied_Combined', 'ACCOUNT',
       'ACCOUNT_Cultural', 'ACCOUNT_Individual', 'ACCOUNT_Other',
       'COMMUNITYRECOVERY', 'EVENT', 'GRIEF', 'GRIEF_Individual',
       'GRIEF_Community', 'GRIEF_Societal', 'HERO', 'INVESTIGATION', 'JOURNEY',
       'JOURNEY_Mental', 'JOURNEY_Physical', 'LEGAL', 'MEDIA', 'MISCELLANEOUS',
       'MOURNING', 'MOURNING_Individual', 'MOURNING_Community',
       'MOURNING_Societal', 'PERPETRATOR', 'PHOTO', 'POLICY', 'POLICY_Guns',
       'POLICY_InfoSharing', 'POLICY_MentalHealth', 'POLICY_Other',
       'POLICY_VictimAdv', 'POLICY_OtherAdv', 'POLICY_Practice',
       'PRIVATESECTOR', 'RACECULTURE', 'RESOURCES', 'SAFETY',
       'SAFETY_Community', 'SAFETY_Individual', 'SAFETY_SchoolOrg',
       'SAFETY_Societal', 'SOCIALSUPPORT', 'THREAT', 'THREAT_Assessment',
       'TRAUMA', 'TRAUMA_Physical', 'TRAUMA_Psychological',
       'TRAUMA_Individual', 'TRAUMA_Community', 'TRAUMA_Societal', 'VICTIMS',
       'word_

In [6]:
data = data[['StoryID', 'Excerpt', 'Excerpt_pre', 'Excerpt_tokenized',
       'Excerpt_nonstop', 'Excerpt_stemmed', 'Excerpt_lemmatized',
       'Excerpt_clean', 'ACCOUNT_Individual']]

In [7]:
data.shape

(8131, 9)

In [8]:
data.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Individual
0,NI3079,Elliot Rodger Wangs roommate stabbed Wang and ...,elliot rodger wangs roommate stabbed wang two ...,"['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...",0
1,NI2689,Because the tragedy unfolded at the start of a...,tragedy unfolded start holiday weekend school ...,"['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...",0
2,NI2400,Gunrelated restraining orders Also passed in r...,gunrelated restraining orders also passed resp...,"['gunrelated', 'restraining', 'orders', 'also'...","['gunrelated', 'restraining', 'orders', 'also'...","['gunrel', 'restrain', 'order', 'also', 'pass'...","['gunrelated', 'restraining', 'order', 'also',...","['gunrel', 'restrain', 'order', 'also', 'pass'...",0
3,NI1393,Linder said she will also remember Weiss for h...,linder said also remember weiss epic shot pool...,"['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'rememb', 'weiss', ...","['linder', 'said', 'also', 'remember', 'wei', ...","['linder', 'said', 'also', 'rememb', 'weiss', ...",0
4,NI3259,I mean I know how this goes We all do Werent y...,mean know goes werent sort expecting father on...,"['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...","['mean', 'know', 'go', 'werent', 'sort', 'expe...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...",0


In [9]:
data_unin = data.loc[data['ACCOUNT_Individual']==0]
data_unin.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Individual
0,NI3079,Elliot Rodger Wangs roommate stabbed Wang and ...,elliot rodger wangs roommate stabbed wang two ...,"['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...","['elliot', 'rodger', 'wangs', 'roommate', 'sta...","['elliot', 'rodger', 'wang', 'roommat', 'stab'...",0
1,NI2689,Because the tragedy unfolded at the start of a...,tragedy unfolded start holiday weekend school ...,"['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...","['tragedy', 'unfolded', 'start', 'holiday', 'w...","['tragedi', 'unfold', 'start', 'holiday', 'wee...",0
2,NI2400,Gunrelated restraining orders Also passed in r...,gunrelated restraining orders also passed resp...,"['gunrelated', 'restraining', 'orders', 'also'...","['gunrelated', 'restraining', 'orders', 'also'...","['gunrel', 'restrain', 'order', 'also', 'pass'...","['gunrelated', 'restraining', 'order', 'also',...","['gunrel', 'restrain', 'order', 'also', 'pass'...",0
3,NI1393,Linder said she will also remember Weiss for h...,linder said also remember weiss epic shot pool...,"['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'remember', 'weiss'...","['linder', 'said', 'also', 'rememb', 'weiss', ...","['linder', 'said', 'also', 'remember', 'wei', ...","['linder', 'said', 'also', 'rememb', 'weiss', ...",0
4,NI3259,I mean I know how this goes We all do Werent y...,mean know goes werent sort expecting father on...,"['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goes', 'werent', 'sort', 'ex...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...","['mean', 'know', 'go', 'werent', 'sort', 'expe...","['mean', 'know', 'goe', 'werent', 'sort', 'exp...",0


In [10]:
data_unin.ACCOUNT_Individual.unique()

array([0])

In [11]:
data_unin.shape

(7343, 9)

In [12]:
data_in = data.loc[data['ACCOUNT_Individual']==1]
data_in.shape

(788, 9)

In [13]:
data_unin = data_unin.sample(788)
data_unin.shape

(788, 9)

In [14]:
data_in.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Individual
65,NI2577,One thing for sure Every legal expert agrees t...,one thing sure every legal expert agrees rodge...,"['one', 'thing', 'sure', 'every', 'legal', 'ex...","['one', 'thing', 'sure', 'every', 'legal', 'ex...","['one', 'thing', 'sure', 'everi', 'legal', 'ex...","['one', 'thing', 'sure', 'every', 'legal', 'ex...","['one', 'thing', 'sure', 'everi', 'legal', 'ex...",1
73,NI3119,Certainly the latest murderous rampage carried...,certainly latest murderous rampage carried you...,"['certainly', 'latest', 'murderous', 'rampage'...","['certainly', 'latest', 'murderous', 'rampage'...","['certainli', 'latest', 'murder', 'rampag', 'c...","['certainly', 'latest', 'murderous', 'rampage'...","['certainli', 'latest', 'murder', 'rampag', 'c...",1
74,NI1605,In a 137page manifesto Rodger wrote that he wo...,137page manifesto rodger wrote would commit ma...,"['137page', 'manifesto', 'rodger', 'wrote', 'w...","['137page', 'manifesto', 'rodger', 'wrote', 'w...","['137page', 'manifesto', 'rodger', 'wrote', 'w...","['137page', 'manifesto', 'rodger', 'wrote', 'w...","['page', 'manifesto', 'rodger', 'wrote', 'woul...",1
76,NI2957,Some crackpot who couldnt get a date stabs and...,crackpot couldnt get date stabs shoots way acr...,"['crackpot', 'couldnt', 'get', 'date', 'stabs'...","['crackpot', 'couldnt', 'get', 'date', 'stabs'...","['crackpot', 'couldnt', 'get', 'date', 'stab',...","['crackpot', 'couldnt', 'get', 'date', 'stab',...","['crackpot', 'couldnt', 'get', 'date', 'stab',...",1
83,NI2915,I got into an online debate recently with Lind...,got online debate recently lindsay beyerstein ...,"['got', 'online', 'debate', 'recently', 'linds...","['got', 'online', 'debate', 'recently', 'linds...","['got', 'onlin', 'debat', 'recent', 'lindsay',...","['got', 'online', 'debate', 'recently', 'linds...","['got', 'onlin', 'debat', 'recent', 'lindsay',...",1


In [15]:
data_in.ACCOUNT_Individual.unique()

array([1])

In [16]:
dataframe = [data_unin, data_in]
data = pd.concat(dataframe)
data.shape

(1576, 9)

In [17]:
data['Excerpt'].fillna("unknown", inplace=True)

In [18]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,StoryID,Excerpt,Excerpt_pre,Excerpt_tokenized,Excerpt_nonstop,Excerpt_stemmed,Excerpt_lemmatized,Excerpt_clean,ACCOUNT_Individual
1931,NI2699,Before Rodger stabbed three male UCSB students...,rodger stabbed three male ucsb students apartm...,"['rodger', 'stabbed', 'three', 'male', 'ucsb',...","['rodger', 'stabbed', 'three', 'male', 'ucsb',...","['rodger', 'stab', 'three', 'male', 'ucsb', 's...","['rodger', 'stabbed', 'three', 'male', 'ucsb',...","['rodger', 'stab', 'three', 'male', 'ucsb', 's...",1
3092,NI1731,Some crackpot who couldnt get a date stabs and...,crackpot couldnt get date stabs shoots way acr...,"['crackpot', 'couldnt', 'get', 'date', 'stabs'...","['crackpot', 'couldnt', 'get', 'date', 'stabs'...","['crackpot', 'couldnt', 'get', 'date', 'stab',...","['crackpot', 'couldnt', 'get', 'date', 'stab',...","['crackpot', 'couldnt', 'get', 'date', 'stab',...",1
6795,NI2308,The awkward 22yearold was obsessed with exacti...,awkward 22yearold obsessed exacting retributio...,"['awkward', '22yearold', 'obsessed', 'exacting...","['awkward', '22yearold', 'obsessed', 'exacting...","['awkward', '22yearold', 'obsess', 'exact', 'r...","['awkward', '22yearold', 'obsessed', 'exacting...","['awkward', 'yearold', 'obsess', 'exact', 'ret...",1
4605,NI1018,Shifman said the family\nwas cooperating with ...,shifman said family cooperating police read st...,"['shifman', 'said', 'family', 'cooperating', '...","['shifman', 'said', 'family', 'cooperating', '...","['shifman', 'said', 'famili', 'cooper', 'polic...","['shifman', 'said', 'family', 'cooperating', '...","['shifman', 'said', 'famili', 'cooper', 'polic...",0
5716,NI1501,Like other deranged killers many of them young...,like deranged killers many young men psychosex...,"['like', 'deranged', 'killers', 'many', 'young...","['like', 'deranged', 'killers', 'many', 'young...","['like', 'derang', 'killer', 'mani', 'young', ...","['like', 'deranged', 'killer', 'many', 'young'...","['like', 'derang', 'killer', 'mani', 'young', ...",1


In [19]:
data.ACCOUNT_Individual.unique()

array([1, 0])

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1576 entries, 1931 to 7804
Data columns (total 9 columns):
StoryID               1576 non-null object
Excerpt               1576 non-null object
Excerpt_pre           1576 non-null object
Excerpt_tokenized     1576 non-null object
Excerpt_nonstop       1576 non-null object
Excerpt_stemmed       1576 non-null object
Excerpt_lemmatized    1576 non-null object
Excerpt_clean         1576 non-null object
ACCOUNT_Individual    1576 non-null int64
dtypes: int64(1), object(8)
memory usage: 123.1+ KB


## SPLITTING THE DATASET 

In [21]:
df = data['Excerpt']
df.shape

(1576,)

In [22]:
df.head()

1931    Before Rodger stabbed three male UCSB students...
3092    Some crackpot who couldnt get a date stabs and...
6795    The awkward 22yearold was obsessed with exacti...
4605    Shifman said the family\nwas cooperating with ...
5716    Like other deranged killers many of them young...
Name: Excerpt, dtype: object

In [23]:
y = data['ACCOUNT_Individual']
y = y.to_numpy()
y

array([1, 1, 1, ..., 1, 1, 0])

In [24]:
type(y)

numpy.ndarray

In [25]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 276  
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(data, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((1300,), (1300,), (276,), (276,))

In [26]:
X_train.head()

1931    Before Rodger stabbed three male UCSB students...
3092    Some crackpot who couldnt get a date stabs and...
6795    The awkward 22yearold was obsessed with exacti...
4605    Shifman said the family\nwas cooperating with ...
5716    Like other deranged killers many of them young...
Name: Excerpt, dtype: object

In [27]:
X_valid.head()

3275    Elliot Rodger the mentally disturbed Isla Vist...
374     The postmassacre handwringing were currently e...
6754    In rambling writings he titled My Twisted Worl...
3982    Martinezs grief and outrage were so raw in TV ...
7107    And now Elliot Rodger\nAll were young loners w...
Name: Excerpt, dtype: object

In [28]:
y_train

array([1, 1, 1, ..., 0, 1, 1])

In [29]:
y_valid

array([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0])

## MODEL DEPLOYMENT 

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [31]:
import nltk
from nltk.tokenize import TweetTokenizer 
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

In [32]:
n = X_train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(X_train)
test_term_doc = vec.transform(X_valid)

In [33]:
trn_term_doc, test_term_doc

(<1300x12368 sparse matrix of type '<class 'numpy.float64'>'
 	with 159235 stored elements in Compressed Sparse Row format>,
 <276x12368 sparse matrix of type '<class 'numpy.float64'>'
 	with 31382 stored elements in Compressed Sparse Row format>)

In [34]:
trn_doc = trn_term_doc
val_doc = test_term_doc

In [35]:
trn_doc[0]

<1x12368 sparse matrix of type '<class 'numpy.float64'>'
	with 204 stored elements in Compressed Sparse Row format>

In [36]:
vocab = vec.get_feature_names()

In [37]:
vocab[5000:5010]

['hours explaining',
 'house',
 'house and',
 'house as',
 'house bleeding',
 'house he',
 'house i',
 'house ill',
 'house jealousy',
 'house of']

## NAIVE BAYES 

In [39]:
x = trn_doc
x

<1300x12368 sparse matrix of type '<class 'numpy.float64'>'
	with 159235 stored elements in Compressed Sparse Row format>

In [40]:
type(y_train)

numpy.ndarray

In [41]:
y = y_train

In [42]:
y.shape

(1300,)

In [43]:
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1

In [44]:
p

matrix([[1.54882, 1.29205, 1.41781, ..., 1.16195, 1.     , 1.     ]])

In [45]:
p.shape

(1, 12368)

In [46]:
q

matrix([[1.06057, 1.     , 2.51995, ..., 2.14899, 1.54701, 1.54701]])

In [47]:
q.shape

(1, 12368)

In [48]:
r = np.log((p/p.sum())/(q/q.sum()))

In [49]:
r

matrix([[ 0.30726,  0.18481, -0.64654, ..., -0.68632, -0.50775, -0.50775]])

In [50]:
r.shape

(1, 12368)

In [51]:
b = np.log(len(p)/len(q))

In [52]:
b

0.0

In [53]:
type(y_valid)

numpy.ndarray

In [54]:
y_valid.shape

(276,)

In [55]:
val_y = y_valid.reshape(1, 276)
val_y.shape

(1, 276)

In [56]:
pre_preds = val_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.8985507246376812

In [57]:
pre_preds = val_doc.sign() @ r.T + b
preds = pre_preds.T > 0
(preds==val_y).mean()

0.9021739130434783

## LOGISTIC REGRESSION 

In [58]:
x

<1300x12368 sparse matrix of type '<class 'numpy.float64'>'
	with 159235 stored elements in Compressed Sparse Row format>

In [59]:
y

array([1, 1, 1, ..., 0, 1, 1])

In [60]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(x, y)
preds = m.predict(val_doc)
(preds==val_y).mean()

/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8985507246376812

In [61]:
m = LogisticRegression(C=1e8, dual=True)
m.fit(trn_doc.sign(), y)
preds = m.predict(val_doc.sign())
(preds==val_y).mean()

/home/spriyanshu723/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8876811594202898

In [62]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_doc, y)
preds = m.predict(val_doc)
(preds==val_y).mean()

0.8985507246376812

In [63]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_doc.sign(), y)
preds = m.predict(val_doc.sign())
(preds==val_y).mean()

0.894927536231884

## TRI-GRAM MODEL DEPLOYMENT 

In [64]:
n = X_train.shape[0]
vect = TfidfVectorizer(ngram_range=(1,4), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vect.fit_transform(X_train)
test_term_doc = vect.transform(X_valid)

In [65]:
trn_doc = trn_term_doc

In [66]:
val_doc = test_term_doc

In [67]:
trn_doc.shape

(1300, 30870)

In [68]:
vocab = vect.get_feature_names()

In [69]:
vocab[20000:20005]

['readers he was', 'readers he was born', 'ready', 'real', 'realities']

In [70]:
y = y_train
y

array([1, 1, 1, ..., 0, 1, 1])

In [71]:
val_y

array([[1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0]])

In [72]:
x = trn_doc.sign()
x

<1300x30870 sparse matrix of type '<class 'numpy.float64'>'
	with 273716 stored elements in Compressed Sparse Row format>

In [73]:
val_x = val_doc.sign()
val_x

<276x30870 sparse matrix of type '<class 'numpy.float64'>'
	with 50709 stored elements in Compressed Sparse Row format>

In [74]:
p = x[y==1].sum(0)+1
q = x[y==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [75]:
pre_preds = val_x @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.855072463768116

In [76]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x,y)
preds = m.predict(val_x)
(preds==val_y).mean()

0.8804347826086957

In [77]:
r.shape

(1, 30870)

In [78]:
r

matrix([[ 0.59113,  0.87881,  0.87881, ..., -1.89378, -2.11692, -2.11692]])

In [79]:
np.exp(r)

matrix([[1.80603, 2.40804, 2.40804, ..., 0.1505 , 0.1204 , 0.1204 ]])

In [80]:
x_nb = x.multiply(r)
m = LogisticRegression(C=0.1, dual=True)
m.fit(x_nb,y)

val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
(preds==val_y).mean()

0.8985507246376812